## Investigating the Zillow Housing Price Dataset

N. Ranjan & R. Mattson | CSCI 6430* | Mar 11, 2021




TODO: 
- check file names

In [3]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline


###  Retrieve Data

In [10]:
#TODO - get from google docs??

DF1B = pd.read_csv(r'Datasets/City_MedianRentalPrice_1Bedroom.csv')
DF2B = pd.read_csv(r'Datasets/City_MedianRentalPrice_2Bedroom.csv')
DF3B = pd.read_csv(r'Datasets/City_MedianRentalPrice_3Bedroom.csv')
DF4B = pd.read_csv(r'Datasets/City_MedianRentalPrice_4Bedroom.csv')

In [11]:
DF1B.columns

Index(['Unnamed: 0', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank',
       '2010-02', '2010-03', '2010-04', '2010-05',
       ...
       '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
       '2019-09', '2019-10', '2019-11', '2019-12'],
      dtype='object', length=125)

In [20]:
np.sort(DF1B['Metro'].to_numpy())

array(['Abilene', 'Akron', 'Albany-Schenectady-Troy',
       'Albany-Schenectady-Troy', 'Albany-Schenectady-Troy',
       'Albany-Schenectady-Troy', 'Albany-Schenectady-Troy',
       'Albuquerque', 'Allentown-Bethlehem-Easton',
       'Allentown-Bethlehem-Easton', 'Amarillo', 'Ames', 'Anchorage',
       'Ann Arbor', 'Asheville', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-Sandy Springs-Roswell',
       'Atlanta-Sandy Springs-Roswell', 'Atlanta-San

In [180]:
# remove columns with useless information
DF1B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF2B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF3B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF4B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)

In [181]:
# record the number of bedrooms within the frame
DF1B.insert(4, 'Bedrooms', 1)
DF2B.insert(4, 'Bedrooms', 2)
DF3B.insert(4, 'Bedrooms', 3)
DF4B.insert(4, 'Bedrooms', 4)

In [182]:
frames = [DF1B,DF2B,DF3B,DF4B]
result = pd.concat(frames)
result = result.sample(frac=1) #Shuffle!
result = result.reset_index(drop=True) #Reset Index

In [183]:
result

,RegionName,State,Metro,CountyName,Bedrooms,2010-02,2010-03,2010-04,2010-05,2010-06,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,Oshkosh,WI,Oshkosh-Neenah,Winnebago County,2,NaN,NaN,NaN,NaN,NaN,...,725.0,750.0,720.0,700.0,745.0,747.5,725.0,740.0,740.0,725.0
1,Norristown,PA,Philadelphia-Camden-Wilmington,Montgomery County,2,NaN,NaN,NaN,NaN,NaN,...,1397.0,1399.0,1350.0,1360.0,1375.0,1391.0,1382.5,1450.0,1497.5,1475.0
2,Annapolis,MD,Baltimore-Columbia-Towson,Anne Arundel County,1,NaN,NaN,NaN,NaN,NaN,...,1655.0,1650.0,1680.0,1675.0,1675.0,1602.5,1584.0,1562.0,1575.0,1580.0
3,Hutchinson Island South,FL,Port St. Lucie,Saint Lucie County,2,NaN,NaN,NaN,NaN,NaN,...,2950.0,2800.0,2800.0,2800.0,2800.0,2900.0,2800.0,3000.0,3050.0,3200.0
4,Newton,MA,Boston-Cambridge-Newton,Middlesex County,2,NaN,NaN,NaN,NaN,NaN,...,2375.0,2375.0,2400.0,2400.0,2400.0,2500.0,2400.0,2400.0,2400.0,2300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,Keller,TX,Dallas-Fort Worth-Arlington,Tarrant County,3,NaN,NaN,NaN,NaN,NaN,...,1795.0,1892.5,2000.0,1800.0,1740.0,1749.5,1825.0,1697.5,1689.5,1697.0
3385,Columbia,SC,Columbia,Richland County,4,NaN,NaN,NaN,NaN,NaN,...,1400.0,1425.0,1412.5,1364.5,1449.5,1400.0,1300.0,1349.5,1400.0,1400.0
3386,Middletown,OH,Cincinnati,Butler County,2,NaN,NaN,NaN,NaN,NaN,...,755.0,775.0,785.0,795.0,750.0,737.5,790.0,800.0,745.0,750.0
3387,Gainesville,FL,Gainesville,Alachua County,3,NaN,NaN,NaN,NaN,NaN,...,1495.0,1500.0,1479.0,1490.0,1475.0,1490.0,1409.0,1400.0,1425.0,1450.0


In [184]:
# result.to_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
result.to_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv',index=False)



In [185]:
# dels

## Preproccess Data

In [136]:
# DF = pd.read_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
DF = pd.read_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv')

In [137]:
print(DF.isnull().sum().to_numpy())

[   0    0    0    0    0 3365 3352 3348 3346 3346 3337 3328 3306 3287
 3280 3271 3136 3016 2990 2972 2932 2887 2802 2760 2723 2675 2646 2634
 2604 2587 2575 2561 2548 2538 2526 2519 2514 2500 2493 2483 2479 2472
 2462 2459 2456 2452 2452 2448 2445 2431 2419 2419 2402 2388 2385 2384
 2382 2365 2359 2349 2346 2337 2333 2321 2312 2305 2297 2288 2284 2279
 2273 2267 2261 2258 2254 2245 2240 2235 2231 2225 2213 2185 2173 2154
 2121 2084 2020 1907 1400 1206 1109 1047  963  913  867  829  800  771
  741  695  621  571  510  465  412  361  306  249  209  144  107   60
    0    0    0    0    0    0    0    0    0    0    0    0]


In [138]:
cut_off = #why

SyntaxError: invalid syntax (<ipython-input-138-1a4325d0907d>, line 1)

In [139]:
# DF_Filt = DF.iloc[:,cut_off:-1]
DF_Filt = DF.iloc[:,-16:-1]
DF_Filt # Contains all numerical cols we want
for x in DF_Filt.columns.to_numpy():
   #sc = StandardScaler()
   DF_Filt[x] = DF_Filt[x].values.reshape(-1,1)
  
DFLblColumns = DF.iloc[:,0:5]
print(DFLblColumns.columns)
for x in DFLblColumns.columns.to_numpy():
   le = LabelEncoder() 
   DF_Filt[x] = le.fit_transform(DFLblColumns[x].values)

Index(['RegionName', 'State', 'Metro', 'CountyName', 'Bedrooms'], dtype='object')


In [140]:
Y = DF.iloc[:,-1]
Y

0        725.0
1       1475.0
2       1580.0
3       3200.0
4       2300.0
         ...  
3384    1697.0
3385    1400.0
3386     750.0
3387    1450.0
3388    1300.0
Name: 2019-12, Length: 3389, dtype: float64

In [141]:
X = DF_Filt
X

,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,RegionName,State,Metro,CountyName,Bedrooms
0,795.0,725.0,747.5,725.0,712.5,700.0,725.0,750.0,720.0,700.0,745.0,747.5,725.0,740.0,740.0,902,48,208,436,1
1,1445.0,1422.0,1445.0,1355.0,1350.0,1366.0,1397.0,1399.0,1350.0,1360.0,1375.0,1391.0,1382.5,1450.0,1497.5,843,38,214,265,1
2,1680.0,1680.0,1690.0,1615.0,1600.0,1630.0,1655.0,1650.0,1680.0,1675.0,1675.0,1602.5,1584.0,1562.0,1575.0,27,20,19,10,0
3,2800.0,2800.0,3000.0,3200.0,3200.0,3000.0,2950.0,2800.0,2800.0,2800.0,2800.0,2900.0,2800.0,3000.0,3050.0,556,9,219,351,1
4,2300.0,2250.0,2300.0,2300.0,2222.5,2250.0,2375.0,2375.0,2400.0,2400.0,2400.0,2500.0,2400.0,2400.0,2400.0,835,19,29,256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,1697.5,1662.5,1700.0,1895.0,1834.0,1745.0,1795.0,1892.5,2000.0,1800.0,1740.0,1749.5,1825.0,1697.5,1689.5,592,43,66,400,2
3385,1295.0,1350.0,1400.0,1375.0,1399.0,1375.0,1400.0,1425.0,1412.5,1364.5,1449.5,1400.0,1300.0,1349.5,1400.0,235,40,61,332,3
3386,735.0,725.0,725.0,725.0,735.0,735.0,755.0,775.0,785.0,795.0,750.0,737.5,790.0,800.0,745.0,755,35,54,50,1
3387,1350.0,1400.0,1400.0,1400.0,1475.0,1500.0,1495.0,1500.0,1479.0,1490.0,1475.0,1490.0,1409.0,1400.0,1425.0,434,9,101,2,2


In [146]:
print("Number of nulls per column", X.isnull().sum())
for col in X:
    X.fillna(X[col].median(),inplace=True)
    #X = X.fillna(X[col].median())
    
X_full = X.copy()
print("After Replacing Null:")   
print("Max amount on nulls per column", X.isnull().sum().max())

Number of nulls per column 2018-09     0
2018-10     0
2018-11     0
2018-12     0
2019-01     0
2019-02     0
2019-03     0
2019-04     0
2019-05     0
2019-06     0
2019-07     0
2019-08     0
2019-09     0
2019-10     0
2019-11     0
Bedrooms    0
dtype: int64
After Replacing Null:
Max amount on nulls per column 0


## Run Models

### Linear Regression

In [123]:
def show_linear_coefs():
    # Peek at learned parameters
    model_temp = LinearRegression() 
    hist = model_temp.fit(X,Y)
    DF = pd.DataFrame(X.columns)
    DF['parameters '] = model_temp.coef_
    print("\n\nLearned Coefficients")
    print(DF)
    return

In [127]:
def run_regression():
    model = make_pipeline(StandardScaler(),LinearRegression())
    scores = cross_validate(model, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    
    meanR2 = np.absolute(scores['test_r2']).mean()
    medianR2 = np.median(np.absolute(scores['test_r2']))
    meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
    medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
    meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
    medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
    print('Average R2 score ', meanR2)
    print('Median R2 score ', medianR2)
    print('Average Root Mean Square Error ', meanRMSE)
    print('Median Root Mean Square Error ', medianRMSE)
    print('Average Mean Absolute Percentage Error ', meanMAPE)
    print('Median Mean Absolute Percentage Error ', medianMAPE)
    show_linear_coefs()
    return

In [128]:
run_regression()

Average R2 score  0.9932104174998116
Median R2 score  0.993506237546615
Average Root Mean Square Error  141.10762758543027
Median Root Mean Square Error  106.22689409783752
Average Mean Absolute Percentage Error  0.03048792778833856
Median Mean Absolute Percentage Error  0.03101421765014442


Learned Coefficients
             0  parameters 
0      2018-09    -0.029692
1      2018-10     0.056345
2      2018-11     0.040309
3      2018-12     0.045587
4      2019-01    -0.201419
5      2019-02     0.278329
6      2019-03     0.113017
7      2019-04    -0.037638
8      2019-05     0.055304
9      2019-06    -0.425628
10     2019-07     0.077633
11     2019-08     0.197798
12     2019-09     0.238308
13     2019-10    -0.122747
14     2019-11     0.752773
15  RegionName    -0.000978
16       State     0.482018
17       Metro    -0.048513
18  CountyName    -0.005333
19    Bedrooms    -8.351810


Some parameter are producing very small parameter values, indicating that nothing useful is being learned.... I protest this 

In [129]:
X.drop(columns=['RegionName', 'CountyName'], inplace=True)
run_regression()

Average R2 score  0.9932239724624937
Median R2 score  0.9935110670427175
Average Root Mean Square Error  140.9705334472767
Median Root Mean Square Error  105.98058943126352
Average Mean Absolute Percentage Error  0.030444065100653162
Median Mean Absolute Percentage Error  0.030996405852025313


Learned Coefficients
           0  parameters 
0    2018-09    -0.029669
1    2018-10     0.056100
2    2018-11     0.040320
3    2018-12     0.045789
4    2019-01    -0.201198
5    2019-02     0.278062
6    2019-03     0.113175
7    2019-04    -0.037557
8    2019-05     0.055440
9    2019-06    -0.425767
10   2019-07     0.077572
11   2019-08     0.197647
12   2019-09     0.238579
13   2019-10    -0.122893
14   2019-11     0.752656
15     State     0.483209
16     Metro    -0.051738
17  Bedrooms    -8.348131


wait yeah dropping useless columns did nothing duh. 

Drop all categorical data? 

In [130]:
X.drop(columns=['Metro', 'State'], inplace=True)
run_regression()

Average R2 score  0.9932124123319527
Median R2 score  0.9936033212211287
Average Root Mean Square Error  141.05547532118254
Median Root Mean Square Error  107.05327595292545
Average Mean Absolute Percentage Error  0.030408083425086635
Median Mean Absolute Percentage Error  0.030805314820183154


Learned Coefficients
           0  parameters 
0    2018-09    -0.030528
1    2018-10     0.057949
2    2018-11     0.038883
3    2018-12     0.045958
4    2019-01    -0.200494
5    2019-02     0.281652
6    2019-03     0.111426
7    2019-04    -0.035360
8    2019-05     0.058768
9    2019-06    -0.430052
10   2019-07     0.082321
11   2019-08     0.197657
12   2019-09     0.224853
13   2019-10    -0.116803
14   2019-11     0.750615
15  Bedrooms    -7.966284


I got no improvement from droppin all categorical data. its already pretty good. .... nikhil wym??? 

### Polynomial Interpolation

In [148]:
X = X_full.copy()

In [144]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

for _,degree in enumerate([1,2,3]):
    polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Ridge(alpha=1e-3))
    scores = cross_validate(polymodel, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    print('-'*5)
    print('Polynomial Degree: '+str(degree))
    print('-'*5)
#     meanR2 = np.absolute(scores['test_r2']).mean()
#     medianR2 = np.median(np.absolute(scores['test_r2']))
    meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
    medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
    meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
    medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
#     print('Average R2 score ',meanR2)
#     print('Median R2 score ',medianR2)
    print('Average Root Mean Square Error ',meanRMSE)
    print('Median Root Mean Square Error ',medianRMSE)
    print('Average Mean Absolute Percentage Error ',meanMAPE)
    print('Median Mean Absolute Percentage Error ',medianMAPE)    
    print('-'*10)
    

-----
Polynomial Degree: 1
-----
Average R2 score  0.9932118431450961
Median R2 score  0.993506267239761
Average Root Mean Square Error  141.09267915084382
Median Root Mean Square Error  106.22235871657745
Average Mean Absolute Percentage Error  0.03048725058340056
Median Mean Absolute Percentage Error  0.031014071969407423
----------
-----
Polynomial Degree: 2
-----
Average R2 score  0.78753550361516
Median R2 score  0.9307205309656819
Average Root Mean Square Error  846.1825789529961
Median Root Mean Square Error  595.3241090145788
Average Mean Absolute Percentage Error  0.02667639335504545
Median Mean Absolute Percentage Error  0.026681146593309567
----------
-----
Polynomial Degree: 3
-----
Average R2 score  531.5687791035417
Median R2 score  17.71572628391656
Average Root Mean Square Error  26565.515046934626
Median Root Mean Square Error  7973.592958371435
Average Mean Absolute Percentage Error  0.07331396766528288
Median Mean Absolute Percentage Error  0.0449919722124477
-------

In [ ]:
## WE should not be doing R^2 score, should we?

In [145]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X.drop(columns=['CountyName','RegionName','Metro', 'State'], inplace=True)
for _,degree in enumerate([1,2,3]):
    polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Ridge(alpha=1e-3))
    scores = cross_validate(polymodel, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    print('-'*5)
    print('Polynomial Degree: '+str(degree))
    print('-'*5)
#     meanR2 = np.absolute(scores['test_r2']).mean()
#     medianR2 = np.median(np.absolute(scores['test_r2']))
    meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
    medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
    meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
    medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
#     print('Average R2 score ',meanR2)
#     print('Median R2 score ',medianR2)
    print('Average Root Mean Square Error ',meanRMSE)
    print('Median Root Mean Square Error ',medianRMSE)
    print('Average Mean Absolute Percentage Error ',meanMAPE)
    print('Median Mean Absolute Percentage Error ',medianMAPE)    
    print('-'*10)
    

-----
Polynomial Degree: 1
-----
Average R2 score  0.9932138559480557
Median R2 score  0.993603335676336
Average Root Mean Square Error  141.04045951327853
Median Root Mean Square Error  107.04802429994103
Average Mean Absolute Percentage Error  0.0304073984403133
Median Mean Absolute Percentage Error  0.03080516161575434
----------
-----
Polynomial Degree: 2
-----
Average R2 score  0.8721595797466781
Median R2 score  0.9649193790988893
Average Root Mean Square Error  813.4179797186036
Median Root Mean Square Error  416.40212608851755
Average Mean Absolute Percentage Error  0.025454089240173155
Median Mean Absolute Percentage Error  0.025078373354244086
----------
-----
Polynomial Degree: 3
-----
Average R2 score  356.6910159106502
Median R2 score  24.24648413165005
Average Root Mean Square Error  22914.25685027387
Median Root Mean Square Error  7625.263005916353
Average Mean Absolute Percentage Error  0.06312640150830037
Median Mean Absolute Percentage Error  0.04332773403985189
-----

This proj is nonsense. 